# QMC Agent - Jupyter Notebook

Este notebook permite probar el agente de scraping de QMC celda por celda.

## Stack Tecnológico
- **Orquestación**: LangGraph
- **Scraping**: Playwright
- **LLM**: Groq (LLaMA 3.3 70B)
- **Framework LLM**: langchain_groq

## 1. Instalación de Dependencias

In [1]:
# Instalar dependencias (ejecutar solo una vez)
!pip install langgraph langchain-groq langchain-core playwright python-dotenv pydantic nest-asyncio
!playwright install chromium

## 2. Configuración del Event Loop (IMPORTANTE para Windows)

In [2]:
# IMPORTANTE: Ejecutar esta celda PRIMERO antes de cualquier código async
# Esto soluciona el problema de NotImplementedError con Playwright en Windows/Jupyter
import nest_asyncio
nest_asyncio.apply()

import asyncio
import sys

# Configuración adicional para Windows
if sys.platform == 'win32':
    asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())

print("✅ Event loop configurado correctamente para Windows/Jupyter")

✅ Event loop configurado correctamente para Windows/Jupyter


## 3. Configuración y Validación

In [3]:
# Cargar configuración
import sys
sys.path.insert(0, '.')

from src.config import Config

# Validar configuración
missing = Config.validate()
if missing:
    print(f"⚠️ Configuración incompleta. Faltan: {', '.join(missing)}")
    print("\n📝 Por favor, completa el archivo .env con los valores requeridos.")
else:
    print("✅ Configuración válida!")
    print(f"   URL: {Config.QMC_URL}")
    print(f"   Usuario: {Config.QMC_USERNAME}")
    print(f"   Modelo LLM: {Config.GROQ_MODEL}")
    print(f"   Headless: {Config.HEADLESS}")

✅ Configuración válida!
   URL: https://apqs.grupoefe.pe/qmc/tasks
   Usuario: srvlimprdap011\consultorqs1
   Modelo LLM: llama-3.3-70b-versatile
   Headless: True


## 4. Test: Nodo de Login

In [4]:
from src.state import create_initial_state
from src.nodes import login_node

# Crear estado inicial
state = create_initial_state()
print("Estado inicial:")
print(f"  current_step: {state['current_step']}")
print(f"  retry_count: {state['retry_count']}")

Estado inicial:
  current_step: init
  retry_count: 0


In [5]:
# Ejecutar login
print("🔐 Ejecutando login...")
login_result = await login_node(state)

print(f"\nResultado:")
print(f"  current_step: {login_result.get('current_step')}")
print(f"  error: {login_result.get('error_message')}")
print(f"  cookies: {'✅' if login_result.get('session_cookies') else '❌'}")

# Actualizar estado
state.update(login_result)

# Mostrar logs
for log in login_result.get('logs', []):
    print(log)

🔐 Ejecutando login...


NotImplementedError: 

## 5. Test: Nodo de Filtrado

In [ ]:
from src.nodes import filter_node

# Solo ejecutar si login fue exitoso
if state.get('current_step') == 'filter':
    print("🔍 Aplicando filtros...")
    filter_result = await filter_node(state)
    
    print(f"\nResultado:")
    print(f"  current_step: {filter_result.get('current_step')}")
    print(f"  error: {filter_result.get('error_message')}")
    print(f"  page_html: {len(filter_result.get('page_html', '')) } chars")
    
    state.update(filter_result)
    
    for log in filter_result.get('logs', []):
        print(log)
else:
    print(f"⚠️ No se puede filtrar. Estado actual: {state.get('current_step')}")

## 6. Test: Nodo de Extracción

In [ ]:
from src.nodes import extract_node

if state.get('current_step') == 'extract':
    print("📊 Extrayendo tabla...")
    extract_result = await extract_node(state)
    
    print(f"\nResultado:")
    print(f"  current_step: {extract_result.get('current_step')}")
    print(f"  raw_table_data: {len(extract_result.get('raw_table_data', ''))} chars")
    
    state.update(extract_result)
    
    # Preview de datos
    if extract_result.get('raw_table_data'):
        print("\n📋 Preview (primeros 500 chars):")
        print(extract_result['raw_table_data'][:500])
else:
    print(f"⚠️ No se puede extraer. Estado actual: {state.get('current_step')}")

## 7. Test: Nodo Analista (LLM)

In [ ]:
from src.analyst import analyst_node
import json

if state.get('current_step') == 'analyze':
    print("🤖 Procesando con LLM (Groq)...")
    analyst_result = await analyst_node(state)
    
    print(f"\nResultado:")
    print(f"  current_step: {analyst_result.get('current_step')}")
    print(f"  error: {analyst_result.get('error_message')}")
    
    state.update(analyst_result)
    
    if analyst_result.get('structured_data'):
        print("\n📄 Datos estructurados:")
        print(json.dumps(analyst_result['structured_data'], indent=2, ensure_ascii=False))
else:
    print(f"⚠️ No se puede analizar. Estado actual: {state.get('current_step')}")

## 8. Ejecución Completa del Grafo

In [ ]:
from src.graph import run_agent
import json

print("🚀 Ejecutando flujo completo...")
print("="*50)

try:
    final_state = await run_agent()
    
    print("\n" + "="*50)
    print("📊 RESULTADO FINAL")
    print("="*50)
    
    print(f"\nEstado: {final_state.get('current_step')}")
    
    if final_state.get('structured_data'):
        print(f"\n✅ {len(final_state['structured_data'])} tareas extraídas:")
        print(json.dumps(final_state['structured_data'], indent=2, ensure_ascii=False))
    
    if final_state.get('error_message'):
        print(f"\n❌ Error: {final_state['error_message']}")
    
    if final_state.get('screenshots'):
        print(f"\n📸 Screenshots guardados: {final_state['screenshots']}")
        
except Exception as e:
    print(f"\n❌ Error de ejecución: {e}")

## 9. Visualizar el Grafo

In [ ]:
from src.graph import build_graph
from IPython.display import Image, display

# Construir y visualizar grafo
workflow = build_graph()
compiled = workflow.compile()

try:
    # Generar imagen del grafo
    graph_image = compiled.get_graph().draw_mermaid_png()
    display(Image(graph_image))
except Exception as e:
    print(f"No se pudo generar imagen del grafo: {e}")
    print("\nUsa el diagrama Mermaid en el implementation_plan.md")

## 10. Exportar Resultados

In [ ]:
import json
from datetime import datetime

if 'final_state' in dir() and final_state.get('structured_data'):
    # Exportar a JSON
    output_file = f"qmc_tasks_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump({
            "extracted_at": datetime.now().isoformat(),
            "tasks": final_state['structured_data']
        }, f, indent=2, ensure_ascii=False)
    
    print(f"✅ Resultados exportados a: {output_file}")
else:
    print("⚠️ No hay datos para exportar. Ejecuta el flujo completo primero.")